Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [ ]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
def logistic_regression_with_L2(beta=0.01, batch_size=128, num_steps=3001, verbose=False):
    
    #beta = 2*sigma = 2*alpha = 2*l2_penalty = 2/C
    
    graph = tf.Graph()
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      logits = tf.matmul(tf_train_dataset, weights) + biases
      print(tf_train_dataset)
      print(weights)
      print(biases)
      print(logits)
    
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 

      # Loss function using L2 Regularization
      regularizer = tf.nn.l2_loss(weights)
      loss = tf.reduce_mean(loss + beta * regularizer)
        
      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
      test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      #print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0 and verbose == True):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    
      print("Logistic Regression with beta=%.4f (l2_penalty: %.6f)" % (beta, beta/2))
      print("Loss at step %d: %f" % (step, l))
      print("Training accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%\n" % accuracy(test_prediction.eval(), test_labels))
      return weights, biases

In [8]:
import time
start_time = time.time()
beta_list=[0, 0.1, 0.01, 0.001]
for beta in beta_list:
    weigths, biases = logistic_regression_with_L2(beta)
print("--- %s seconds ---" % (time.time() - start_time))

Logistic Regression with beta=0.0000 (l2_penalty: 0.000000)
Loss at step 3000: 0.732935
Training accuracy: 86.7%
Validation accuracy: 79.1%
Test accuracy: 86.2%

Logistic Regression with beta=0.1000 (l2_penalty: 0.050000)
Loss at step 3000: 0.917060
Training accuracy: 84.4%
Validation accuracy: 78.8%
Test accuracy: 85.1%

Logistic Regression with beta=0.0100 (l2_penalty: 0.005000)
Loss at step 3000: 0.609562
Training accuracy: 86.7%
Validation accuracy: 81.5%
Test accuracy: 88.2%

Logistic Regression with beta=0.0010 (l2_penalty: 0.000500)
Loss at step 3000: 0.606459
Training accuracy: 86.7%
Validation accuracy: 82.2%
Test accuracy: 88.8%

--- 34.8192760944 seconds ---


In [6]:
import time
# H es hidden nodes, i,e. nodos de la hidden layer

def nn_with_L2(beta=0.01, H=1024, batch_size=128, num_steps=3001, verbose=False):

    #beta = lambda = alpha = l2_penalty = 1/C
    
    graph = tf.Graph()
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)


        # Variables.
        weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, H]))
        biases1 = tf.Variable(tf.zeros([H]))
        weights = tf.Variable(tf.truncated_normal([H, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
        hidden1 = tf.nn.relu(logits1)
        logits = tf.matmul(hidden1, weights) + biases
        # Normal loss function
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
        loss1 = loss
        
        # Loss function with L2 Regularization with beta=0.01
        regularizers = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights)
        loss = tf.reduce_mean(loss + beta * regularizers)

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training
        train_prediction = tf.nn.softmax(logits)

        # Predictions for validation 
        logits1 = tf.matmul(tf_valid_dataset, weights1) + biases1
        hidden1 = tf.nn.relu(logits1)
        logits = tf.matmul(hidden1, weights) + biases
        valid_prediction = tf.nn.softmax(logits)

        # Predictions for test
        logits1 = tf.matmul(tf_test_dataset, weights1) + biases1
        hidden1 = tf.nn.relu(logits1)
        logits = tf.matmul(hidden1, weights) + biases
        test_prediction =  tf.nn.softmax(logits)

    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions, l1, regul = session.run([optimizer, loss, train_prediction, loss1, regularizers], 
                                                       feed_dict=feed_dict)
            if (step % 500 == 0):
                if(verbose == True):
                    print("Minibatch total loss at step {}: {}".format(step, l))
                    print("Loss:{} L2_penalty:{}".format(l1, beta*regul))
                    print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
                    print("Validation accuracy: {:.1f}\n".format(accuracy(valid_prediction.eval(), valid_labels)))
                #time.sleep( 30 )

        print("NN with l2_penalty beta=%.4f" % beta)
        print("Total loss at step %d: %f" % (step, l))
        print("Loss:{} L2_penalty:{}".format(l1, beta*regul))
        print("Training accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%\n" % accuracy(test_prediction.eval(), test_labels))
        return None

In [12]:
import time
start_time = time.time()
beta_list=[0, 0.1, 0.01, 0.001]
for beta in beta_list:
    time.sleep(30)
    nn_with_L2(beta)
print("--- %s seconds ---" % (time.time() - start_time))

NN with l2_penalty beta=0.0000
Total loss at step 3000: 3.454535
Loss:3.45453500748 L2_penalty:0.0
Training accuracy: 82.8%
Validation accuracy: 81.5%
Test accuracy: 88.1%

NN with l2_penalty beta=0.1000
Total loss at step 3000: 1.304930
Loss:0.798663139343 L2_penalty:0.506266975403
Training accuracy: 78.9%
Validation accuracy: 76.4%
Test accuracy: 82.5%

NN with l2_penalty beta=0.0100
Total loss at step 3000: 0.646742
Loss:0.47145229578 L2_penalty:0.175289955139
Training accuracy: 87.5%
Validation accuracy: 83.7%
Test accuracy: 89.8%

NN with l2_penalty beta=0.0010
Total loss at step 3000: 15.419493
Loss:0.385533750057 L2_penalty:15.0339589844
Training accuracy: 90.6%
Validation accuracy: 87.0%
Test accuracy: 92.5%

--- 1343.40050602 seconds ---


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [11]:
import time
start_time = time.time()
train_dataset_orig = train_dataset
train_labels_orig = train_labels
train_dataset = train_dataset[0:300]
train_labels = train_labels[0:300]
#time.sleep(30)
nn_with_L2(beta=0.001, verbose=True)
train_dataset = train_dataset_orig
train_labels = train_labels_orig
print("--- %s seconds ---" % (time.time() - start_time))

Minibatch total loss at step 0: 725.608032227
Loss:410.717651367 L2_penalty:314.89040625
Minibatch accuracy: 10.2
Validation accuracy: 32.4

Minibatch total loss at step 500: 191.369979858
Loss:0.352647274733 L2_penalty:191.017328125
Minibatch accuracy: 99.2
Validation accuracy: 71.5

Minibatch total loss at step 1000: 115.879745483
Loss:0.04653127864 L2_penalty:115.833210937
Minibatch accuracy: 99.2
Validation accuracy: 73.2

Minibatch total loss at step 1500: 70.2469329834
Loss:6.92154890203e-06 L2_penalty:70.246921875
Minibatch accuracy: 100.0
Validation accuracy: 73.0

Minibatch total loss at step 2000: 42.6027641296
Loss:0.00114662304986 L2_penalty:42.6016132813
Minibatch accuracy: 100.0
Validation accuracy: 73.2

Minibatch total loss at step 2500: 25.8372364044
Loss:0.00115957693197 L2_penalty:25.8360761719
Minibatch accuracy: 100.0
Validation accuracy: 73.4

Minibatch total loss at step 3000: 15.6701278687
Loss:0.00141573080327 L2_penalty:15.6687109375
Minibatch accuracy: 100.0


In [7]:
#DAF understanding: offset + repeated batches (i.e. few *different* batches) limiting train_dataset_size=300
#problem 3.2:  to better appreciate the overfitting case, just reduce the size of the training set.
#Either you subset the training set, or cheat the formula by specifying a smaller size of the training set. 
#That will force it to always pick up batches from a smaller training subset. 

batch_size=128
train_dataset_size=300
for step in range(3001):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_dataset_size - batch_size) # (step * batch_size)%172
    # Generate a minibatch.
    print("batch_data = train_dataset[{}:{}, :]".format(offset, offset + batch_size))
    if(step == 50): break

batch_data = train_dataset[0:128, :]
batch_data = train_dataset[128:256, :]
batch_data = train_dataset[84:212, :]
batch_data = train_dataset[40:168, :]
batch_data = train_dataset[168:296, :]
batch_data = train_dataset[124:252, :]
batch_data = train_dataset[80:208, :]
batch_data = train_dataset[36:164, :]
batch_data = train_dataset[164:292, :]
batch_data = train_dataset[120:248, :]
batch_data = train_dataset[76:204, :]
batch_data = train_dataset[32:160, :]
batch_data = train_dataset[160:288, :]
batch_data = train_dataset[116:244, :]
batch_data = train_dataset[72:200, :]
batch_data = train_dataset[28:156, :]
batch_data = train_dataset[156:284, :]
batch_data = train_dataset[112:240, :]
batch_data = train_dataset[68:196, :]
batch_data = train_dataset[24:152, :]
batch_data = train_dataset[152:280, :]
batch_data = train_dataset[108:236, :]
batch_data = train_dataset[64:192, :]
batch_data = train_dataset[20:148, :]
batch_data = train_dataset[148:276, :]
batch_data = train_dataset[104:232, :]


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
import time

def nn_with_L2_and_Dropout(beta=0.01, H=1024, batch_size=128, num_steps=3001, 
                           keep_prob=None, learning_rate=0.5, verbose=False):

    #beta = lambda = alpha = l2_penalty = 1/C
    
    graph = tf.Graph()
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)


        # Variables.
        weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, H], 
                                                   stddev=np.sqrt(2.0 /float(image_size * image_size))))
        biases1 = tf.Variable(tf.zeros([H])) #DAF: According to a student, biases must be zero if we apply xavier_init
        
        weights = tf.Variable(tf.truncated_normal([H, num_labels], stddev=np.sqrt(2.0 /float(H))))
        biases = tf.Variable(tf.zeros([num_labels])) 

        # Training computation.
        logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
        
        if keep_prob is None:
            hidden1 = tf.nn.relu(logits1)
        else:
            hidden1 = tf.nn.dropout(tf.nn.relu(logits1), keep_prob)
            
        logits = tf.matmul(hidden1, weights) + biases
        
        
        
        # Normal loss function
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
        loss1 = loss
        
        # Loss function with L2 Regularization with beta=0.01
        regularizers = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights)
        loss = tf.reduce_mean(loss + beta * regularizers)

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

        # Predictions for the training
        train_prediction = tf.nn.softmax(logits)

        # Predictions for validation 
        logits1 = tf.matmul(tf_valid_dataset, weights1) + biases1
        hidden1 = tf.nn.relu(logits1)
        logits = tf.matmul(hidden1, weights) + biases
        valid_prediction = tf.nn.softmax(logits)

        # Predictions for test
        logits1 = tf.matmul(tf_test_dataset, weights1) + biases1
        hidden1 = tf.nn.relu(logits1)
        logits = tf.matmul(hidden1, weights) + biases
        test_prediction =  tf.nn.softmax(logits)

    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions, l1, regul = session.run([optimizer, loss, train_prediction, loss1, regularizers], feed_dict=feed_dict)
            if (step % 500 == 0):
                if(verbose == True):
                    print("Minibatch total loss at step {}: {}".format(step, l))
                    print("Loss:{} L2_penalty:{}".format(l1, beta*regul))
                    print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
                    print("Validation accuracy: {:.1f}\n".format(accuracy(valid_prediction.eval(), valid_labels)))
                time.sleep( 30 )

        print("NN with l2_penalty beta=%.4f" % beta)
        print("Total loss at step %d: %f" % (step, l))
        print("Loss:{} L2_penalty:{}".format(l1, beta*regul))
        print("Training accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%\n" % accuracy(test_prediction.eval(), test_labels))
        return None

In [9]:
import time
start_time = time.time()
train_dataset_orig = train_dataset
train_labels_orig = train_labels
train_dataset = train_dataset[0:300]
train_labels = train_labels[0:300]
time.sleep(30)
#nn_with_L2_and_Dropout(beta=0.001, keep_prob=0.5, verbose=True)
nn_with_L2_and_Dropout(beta=0, keep_prob=0.5, verbose=True)
train_dataset = train_dataset_orig
train_labels = train_labels_orig
print("--- %s seconds ---" % (time.time() - start_time))

#DAF: we must test "only Dropout without L2", to observe the result!!!.

Minibatch total loss at step 0: 2.53439116478
Loss:2.53439116478 L2_penalty:0.0
Minibatch accuracy: 9.4
Validation accuracy: 37.9

Minibatch total loss at step 500: 0.0349493511021
Loss:0.0349493511021 L2_penalty:0.0
Minibatch accuracy: 99.2
Validation accuracy: 77.9

Minibatch total loss at step 1000: 0.0293263234198
Loss:0.0293263234198 L2_penalty:0.0
Minibatch accuracy: 99.2
Validation accuracy: 77.8

Minibatch total loss at step 1500: 0.000675816379953
Loss:0.000675816379953 L2_penalty:0.0
Minibatch accuracy: 100.0
Validation accuracy: 77.9

Minibatch total loss at step 2000: 0.000989489024505
Loss:0.000989489024505 L2_penalty:0.0
Minibatch accuracy: 100.0
Validation accuracy: 77.7

Minibatch total loss at step 2500: 0.000992382061668
Loss:0.000992382061668 L2_penalty:0.0
Minibatch accuracy: 100.0
Validation accuracy: 77.9

Minibatch total loss at step 3000: 0.00130455684848
Loss:0.00130455684848 L2_penalty:0.0
Minibatch accuracy: 100.0
Validation accuracy: 77.9

NN with l2_penalty

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [8]:
import math
import time
start_time = time.time()

batch_size = 128
beta = 0.001

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))
hidden_nodes_5 = int(hidden_nodes_1 * np.power(0.5, 4))

graph = tf.Graph()
with graph.as_default():

    '''Input Data'''
    # For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    '''Variables'''
    # Hidden RELU layer 1
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=math.sqrt(2.0/(image_size*image_size))))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))

    # Hidden RELU layer 2
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=math.sqrt(2.0/hidden_nodes_1)))
    biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
    # Hidden RELU layer 3
    weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=math.sqrt(2.0/hidden_nodes_2)))
    biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    
    # Hidden RELU layer 4
    weights_4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, hidden_nodes_4], stddev=math.sqrt(2.0/hidden_nodes_3)))
    biases_4 = tf.Variable(tf.zeros([hidden_nodes_4]))
    
    # Hidden RELU layer 5
    weights_5 = tf.Variable(tf.truncated_normal([hidden_nodes_4, hidden_nodes_5], stddev=math.sqrt(2.0/hidden_nodes_4)))
    biases_5 = tf.Variable(tf.zeros([hidden_nodes_5]))
    
    # Output layer
    weights_6 = tf.Variable(tf.truncated_normal([hidden_nodes_5, num_labels], stddev=math.sqrt(2.0/hidden_nodes_5)))
    biases_6 = tf.Variable(tf.zeros([num_labels]))
    
    '''Training computation'''
    
    # Hidden RELU layer 1
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer_1 = tf.nn.relu(logits_1)
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    hidden_layer_1_dropout = tf.nn.dropout(hidden_layer_1, keep_prob)
    
    
    # Hidden RELU layer 2
    logits_2 = tf.matmul(hidden_layer_1_dropout, weights_2) + biases_2
    hidden_layer_2 = tf.nn.relu(logits_2)
    # Dropout on hidden layer: RELU layer
    hidden_layer_2_dropout = tf.nn.dropout(hidden_layer_2, keep_prob)
    
    # Hidden RELU layer 3
    logits_3 = tf.matmul(hidden_layer_2_dropout, weights_3) + biases_3
    hidden_layer_3 = tf.nn.relu(logits_3)
    # Dropout on hidden layer: RELU layer
    hidden_layer_3_dropout = tf.nn.dropout(hidden_layer_3, keep_prob)
    
    # Hidden RELU layer 4
    logits_4 = tf.matmul(hidden_layer_3_dropout, weights_4) + biases_4
    hidden_layer_4 = tf.nn.relu(logits_4)
    # Dropout on hidden layer: RELU layer

    hidden_layer_4_dropout = tf.nn.dropout(hidden_layer_4, keep_prob)
    
    # Hidden RELU layer 5
    logits_5 = tf.matmul(hidden_layer_4_dropout, weights_5) + biases_5
    hidden_layer_5 = tf.nn.relu(logits_5)
    # Dropout on hidden layer: RELU layer
    hidden_layer_5_dropout = tf.nn.dropout(hidden_layer_5, keep_prob)
    
    # Output layer
    logits_6 = tf.matmul(hidden_layer_5_dropout, weights_6) + biases_6 
    
    # Normal loss function
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_6, tf_train_labels))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_6, labels=tf_train_labels))
    # Loss function with L2 Regularization with decaying learning rate beta=0.5
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                   tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + \
                   tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + beta * regularizers)

    '''Optimizer'''
    # Decaying learning rate
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_6)
    
    # Predictions for validation 
    valid_logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    valid_relu_1 = tf.nn.relu(valid_logits_1)
    
    valid_logits_2 = tf.matmul(valid_relu_1, weights_2) + biases_2
    valid_relu_2 = tf.nn.relu(valid_logits_2)
    
    valid_logits_3 = tf.matmul(valid_relu_2, weights_3) + biases_3
    valid_relu_3 = tf.nn.relu(valid_logits_3)
    
    valid_logits_4 = tf.matmul(valid_relu_3, weights_4) + biases_4
    valid_relu_4 = tf.nn.relu(valid_logits_4)
    
    valid_logits_5 = tf.matmul(valid_relu_4, weights_5) + biases_5
    valid_relu_5 = tf.nn.relu(valid_logits_5)
    
    valid_logits_6 = tf.matmul(valid_relu_5, weights_6) + biases_6
    
    valid_prediction = tf.nn.softmax(valid_logits_6)
    
    # Predictions for test
    test_logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    test_relu_1 = tf.nn.relu(test_logits_1)
    
    test_logits_2 = tf.matmul(test_relu_1, weights_2) + biases_2
    test_relu_2 = tf.nn.relu(test_logits_2)
    
    test_logits_3 = tf.matmul(test_relu_2, weights_3) + biases_3
    test_relu_3 = tf.nn.relu(test_logits_3)
    
    test_logits_4 = tf.matmul(test_relu_3, weights_4) + biases_4
    test_relu_4 = tf.nn.relu(test_logits_4)
    
    test_logits_5 = tf.matmul(test_relu_4, weights_5) + biases_5
    test_relu_5 = tf.nn.relu(test_logits_5)
    
    test_logits_6 = tf.matmul(test_relu_5, weights_6) + biases_6
    
    test_prediction = tf.nn.softmax(test_logits_6)
    
    
#################

num_steps = 15000

with tf.Session(graph=graph) as session:
    #tf.initialize_all_variables().run()
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))
    
print("--- %s seconds ---" % (time.time() - start_time))

Initialized
Minibatch loss at step 0: 4.66750097275
Minibatch accuracy: 8.6
Validation accuracy: 10.3
Minibatch loss at step 500: 1.8715518713
Minibatch accuracy: 78.1
Validation accuracy: 83.1
Minibatch loss at step 1000: 1.50912547112
Minibatch accuracy: 82.8
Validation accuracy: 84.4
Minibatch loss at step 1500: 1.08012366295
Minibatch accuracy: 86.7
Validation accuracy: 85.4
Minibatch loss at step 2000: 1.04556286335
Minibatch accuracy: 81.2
Validation accuracy: 85.4
Minibatch loss at step 2500: 0.960202336311
Minibatch accuracy: 82.0
Validation accuracy: 85.0
Minibatch loss at step 3000: 0.964215099812
Minibatch accuracy: 88.3
Validation accuracy: 86.4
Minibatch loss at step 3500: 0.917226314545
Minibatch accuracy: 85.2
Validation accuracy: 83.2
Minibatch loss at step 4000: 0.759147882462
Minibatch accuracy: 87.5
Validation accuracy: 85.6
Minibatch loss at step 4500: 0.719668090343
Minibatch accuracy: 86.7
Validation accuracy: 85.5
Minibatch loss at step 5000: 0.860390543938
Minib

In [7]:
import time
start_time = time.time()

batch_size = 128
hidden_layer1_size = 1024
hidden_layer2_size = 305
hidden_lastlayer_size = 75

use_multilayers = True

regularization_meta=0.03


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  keep_prob = tf.placeholder(tf.float32)

  weights_layer1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer1_size], stddev=0.0517))
  biases_layer1 = tf.Variable(tf.zeros([hidden_layer1_size]))

  if use_multilayers:
    weights_layer2 = tf.Variable(
      tf.truncated_normal([hidden_layer1_size, hidden_layer1_size], stddev=0.0441))
    biases_layer2 = tf.Variable(tf.zeros([hidden_layer1_size]))

    weights_layer3 = tf.Variable(
      tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev=0.0441))
    biases_layer3 = tf.Variable(tf.zeros([hidden_layer2_size]))
    
    weights_layer4 = tf.Variable(
      tf.truncated_normal([hidden_layer2_size, hidden_lastlayer_size], stddev=0.0809))
    biases_layer4 = tf.Variable(tf.zeros([hidden_lastlayer_size]))


  weights = tf.Variable(
    tf.truncated_normal([hidden_lastlayer_size if use_multilayers else hidden_layer1_size, num_labels], stddev=0.1632))
  biases = tf.Variable(tf.zeros([num_labels]))
  
    
  # get the NN models
  def getNN4Layer(dSet, use_dropout):
    input_to_layer1 = tf.matmul(dSet, weights_layer1) + biases_layer1
    hidden_layer1_output = tf.nn.relu(input_to_layer1)
    
    
    logits_hidden1 = None
    if use_dropout:
       dropout_hidden1 = tf.nn.dropout(hidden_layer1_output, keep_prob)
       logits_hidden1 = tf.matmul(dropout_hidden1, weights_layer2) + biases_layer2
    else:
      logits_hidden1 = tf.matmul(hidden_layer1_output, weights_layer2) + biases_layer2
    
    hidden_layer2_output = tf.nn.relu(logits_hidden1)
    
    logits_hidden2 = None
    if use_dropout:
       dropout_hidden2 = tf.nn.dropout(hidden_layer2_output, keep_prob)
       logits_hidden2 = tf.matmul(dropout_hidden2, weights_layer3) + biases_layer3
    else:
      logits_hidden2 = tf.matmul(hidden_layer2_output, weights_layer3) + biases_layer3
    
    
    hidden_layer3_output = tf.nn.relu(logits_hidden2)
    logits_hidden3 = None
    if use_dropout:
       dropout_hidden3 = tf.nn.dropout(hidden_layer3_output, keep_prob)
       logits_hidden3 = tf.matmul(dropout_hidden3, weights_layer4) + biases_layer4
    else:
      logits_hidden3 = tf.matmul(hidden_layer3_output, weights_layer4) + biases_layer4
    
    
    hidden_layer4_output = tf.nn.relu(logits_hidden3)
    logits = None
    if use_dropout:
       dropout_hidden4 = tf.nn.dropout(hidden_layer4_output, keep_prob)
       logits = tf.matmul(dropout_hidden4, weights) + biases
    else:
      logits = tf.matmul(hidden_layer4_output, weights) + biases
    
    return logits

  # get the NN models
  def getNN1Layer(dSet, use_dropout, w1, b1, w, b):
    input_to_layer1 = tf.matmul(dSet, w1) + b1
    hidden_layer1_output = tf.nn.relu(input_to_layer1)
        
    logits = None
    if use_dropout:
       dropout_hidden1 = tf.nn.dropout(hidden_layer1_output, keep_prob)
       logits = tf.matmul(dropout_hidden1, w) + b
    else:
      logits = tf.matmul(hidden_layer1_output, w) + b
    
    return logits

  
  
  # Training computation.
  logits = getNN4Layer(tf_train_dataset, True)  
  logits_valid = getNN4Layer(tf_valid_dataset, False)
  logits_test = getNN4Layer(tf_test_dataset, False)
    
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
  #loss_l2 = loss + (regularization_meta * (tf.nn.l2_loss(weights)))
  
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.3, global_step, 3500, 0.86, staircase=True)
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(logits_valid)
  test_prediction = tf.nn.softmax(logits_test)



num_steps = 95001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.75}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction.eval(feed_dict={tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:1.0}), batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob:1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob:1.0}), test_labels))

print("--- %s seconds ---" % (time.time() - start_time))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0 : 2.43157
Minibatch accuracy: 28.9%
Validation accuracy: 23.7%
Minibatch loss at step 500 : 0.587998
Minibatch accuracy: 88.3%
Validation accuracy: 85.0%
Minibatch loss at step 1000 : 0.407541
Minibatch accuracy: 93.0%
Validation accuracy: 86.7%
Minibatch loss at step 1500 : 0.392963
Minibatch accuracy: 91.4%
Validation accuracy: 87.5%
Minibatch loss at step 2000 : 0.482445
Minibatch accuracy: 90.6%
Validation accuracy: 87.7%
Minibatch loss at step 2500 : 0.386603
Minibatch accuracy: 92.2%
Validation accuracy: 88.3%
Minibatch loss at step 3000 : 0.371642
Minibatch accuracy: 93.0%
Validation accuracy: 88.8%
Minibatch loss at step 3500 : 0.426828
Minibatch accuracy: 94.5%
Validation accuracy: 89.2%
Minibatch loss at step 4000 : 0.280137
Minibatch accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss at step 4500 : 0.34295
Minibatch accuracy: 93.0%
Validation accuracy: 89.6%
